# Sentiment Analysis for Airlines using google/flan-t5-finetuned models

---

***Installing necessary libraries***

In [1]:
!pip install datasets evaluate transformers rouge-score nltk pytesseract tensorboard py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 12.

In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
!pip install nltk

In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


***Login to Huggingface.co***

In [5]:
from huggingface_hub import notebook_login

notebook_login()

***Importing necessary Libraries***

In [6]:
import transformers
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

print(transformers.__version__)
print(np.__version__)
print(pd.__version__)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


4.38.2
1.25.2
2.0.3


In [7]:
# Get multiple outputs in the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# To supress future warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

### Read and PreProcess the data for fine tuning google/flan-t5

In [8]:
inputs_df = pd.read_csv('twitter-airline-sentiment.csv')
inputs_df.head()
inputs_df.tail()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)
14639,569587140490866689,neutral,0.6771,NaN,0.0000,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,2015-02-22 11:58:51 -0800,"dallas, TX",NaN


In [9]:
inputs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [10]:
inputs_df.shape

(14640, 15)

In [11]:
(inputs_df.isnull().sum()/inputs_df.shape[0])*100

tweet_id                         0.000000
airline_sentiment                0.000000
airline_sentiment_confidence     0.000000
negativereason                  37.308743
negativereason_confidence       28.128415
airline                          0.000000
airline_sentiment_gold          99.726776
name                             0.000000
negativereason_gold             99.781421
retweet_count                    0.000000
text                             0.000000
tweet_coord                     93.039617
tweet_created                    0.000000
tweet_location                  32.329235
user_timezone                   32.923497
dtype: float64

***Columns with more than 90% of missing values can be deleted***

In [12]:
print(f'Number of records before deletion is {inputs_df.shape[1]}')
inputs_df.drop(['airline_sentiment_gold','negativereason_gold','tweet_coord'],axis=1, inplace=True)
print(f'Number of records after deletion is {inputs_df.shape[1]}')

Number of records before deletion is 15
Number of records after deletion is 12


In [13]:
inputs_df.airline.nunique()
inputs_df.airline.unique()

6

array(['Virgin America', 'United', 'Southwest', 'Delta', 'US Airways',
       'American'], dtype=object)

In [14]:
airline_df = pd.crosstab(index=inputs_df.airline, columns=inputs_df.airline_sentiment)
airline_df

airline_sentiment,negative,neutral,positive
airline,,,
American,1960,463,336
Delta,955,723,544
Southwest,1186,664,570
US Airways,2263,381,269
United,2633,697,492
Virgin America,181,171,152


In [15]:
airline_normalized_df = round(pd.crosstab(index=inputs_df.airline, columns=inputs_df.airline_sentiment, normalize='index')*100,2)
airline_normalized_df

airline_sentiment,negative,neutral,positive
airline,,,
American,71.04,16.78,12.18
Delta,42.98,32.54,24.48
Southwest,49.01,27.44,23.55
US Airways,77.69,13.08,9.23
United,68.89,18.24,12.87
Virgin America,35.91,33.93,30.16


***Fetching Required Features for Sentiment Analysis***

In [16]:
inputs_df = inputs_df [['text', 'airline_sentiment']]
inputs_df.rename(columns ={'text': 'text', 'airline_sentiment':'label'}, inplace = True)

In [47]:
inputs_df['label'].unique()

array([ 0,  1, -1])

In [46]:
inputs_df['label'] = inputs_df['label'].replace({'neutral': 0, 'negative': -1, 'positive': 1})
inputs_df['label'].unique()

In [17]:
inputs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14640 non-null  object
 1   label   14640 non-null  object
dtypes: object(2)
memory usage: 228.9+ KB


In [18]:
inputs_df.shape

(14640, 2)

In [48]:
inputs_df.head(10)

,text,label
0,@VirginAmerica What @dhepburn said.,0
1,@VirginAmerica plus you've added commercials t...,1
2,@VirginAmerica I didn't today... Must mean I n...,0
3,@VirginAmerica it's really aggressive to blast...,-1
4,@VirginAmerica and it's a really big bad thing...,-1
5,@VirginAmerica seriously would pay $30 a fligh...,-1
6,"@VirginAmerica yes, nearly every time I fly VX...",1
7,@VirginAmerica Really missed a prime opportuni...,0
8,"@virginamerica Well, I didn't…but NOW I DO! :-D",1
9,"@VirginAmerica it was amazing, and arrived an ...",1


***Split the data into Train, Validation and Test dataset***

In [49]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(inputs_df, test_size = 0.4)

X_train, X_test = train_test_split(X_train, test_size = 0.1)

In [50]:
print(f'Shape of X_train is: {X_train.shape}')
print(f'Shape of X_val is: {X_val.shape}')
print(f'Shape of X_test is: {X_test.shape}')

Shape of X_train is: (7905, 2)
Shape of X_val is: (5856, 2)
Shape of X_test is: (879, 2)


***Saving the dataset for training, validation and test***

In [51]:
inputs_df.to_csv('preprocessed_data.csv', index=False)
X_train.to_csv('training_data.csv', index=False)
X_val.to_csv('validation_data.csv', index=False)
X_test.to_csv('test_data.csv', index=False)

***Loading the dataset from Huggingface Dataset***

In [52]:
from datasets import load_dataset

data_files = {"train": "training_data.csv", "validation":"validation_data.csv", "test": "test_data.csv"}
dataset = load_dataset("sudhanshusinghaiml/airlines-sentiment-analysis", data_files= data_files)

Generating train split:   0%|          | 0/7905 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5856 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/879 [00:00<?, ? examples/s]

In [53]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7905
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5856
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 879
    })
})

In [54]:
dataset['train'][29]

{'text': '@AmericanAir ive literally been holding for more than 2 HOURS now. i was told 2hrs. You all are sabotaging any chance i have of getting home',
 'label': -1}

### Tokenize - the training, validation and test dataset

In [55]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)

***Get maximum target length and maximum source length***

Before we can start training we need to preprocess our data. flan t5 is good for a text2text-generation task. This means our model will take a text as input and generate a text as output. For this we want to understand how long our input and output will be to be able to efficiently batch our data.

In [57]:
import pandas as pd
from datasets import Dataset
import random

# dataset['train'] = dataset['train'].shuffle(seed=42).select(range(2000))
# dataset['test'] = dataset['test'].shuffle(seed=42).select(range(1000))

dataset['train'] = dataset['train'].shuffle(seed=42)

train_df = pd.DataFrame(dataset['train'])
validation_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

dataset.clear()

train_df['label'] = train_df['label'].astype(str)
validation_df['label'] = validation_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)

dataset['train'] = Dataset.from_pandas(train_df)
dataset['validation'] = Dataset.from_pandas(validation_df)
dataset['test'] = Dataset.from_pandas(test_df)

In [59]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["validation"]]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["validation"]]).map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/13761 [00:00<?, ? examples/s]

Max source length: 71


Map:   0%|          | 0/13761 [00:00<?, ? examples/s]

Max target length: 3


***Function to pre-process the tokenized dataset***

In [60]:
def preprocess_function(sample, padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [61]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'label'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/7905 [00:00<?, ? examples/s]

Map:   0%|          | 0/5856 [00:00<?, ? examples/s]

Map:   0%|          | 0/879 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [62]:
tokenized_dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7905
})

### ***Fine-tune and evaluate FLAN-T5***

After we have processed our dataset, we can start training our model. First we need to load our FLAN-T5 from the Hugging Face Hub. Then we are using google colab T4 instance to fine-tune the base version of the model. After finetuning the model, we will evaluate it and use the model for inference.

***Loading the pre-trained model***

In [63]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

***Function to Evaluate our finetuned model***

In [64]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("f1")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

***Before we can start training is to create a DataCollator that will take care of padding our inputs and labels. We will use the DataCollatorForSeq2Seq from the 🤗 Transformers library.***

In [65]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

***The last step is to define the hyperparameters (TrainingArguments) we want to use for our training. We are leveraging the Hugging Face Hub integration of the Trainer to automatically push our checkpoints, logs and metrics during training into a repository.***

In [67]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-airline-sentiment-analysis"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-4,
    num_train_epochs=2,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="epoch",
    # logging_steps=1000,
    evaluation_strategy="no",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

***Login to Huggingface.co for write***

In [35]:
from huggingface_hub import notebook_login

notebook_login()

In [68]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
)

In [69]:
# Start training
trainer.train()

Step,Training Loss
989,0.199500
1978,0.098600


Checkpoint destination directory flan-t5-base-airline-sentiment-analysis/checkpoint-989 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory flan-t5-base-airline-sentiment-analysis/checkpoint-1978 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1978, training_loss=0.14905863994293675, metrics={'train_runtime': 558.3305, 'train_samples_per_second': 28.317, 'train_steps_per_second': 3.543, 'total_flos': 1522408182497280.0, 'train_loss': 0.14905863994293675, 'epoch': 2.0})

***Let us Evaluate our finetuned model***

In [70]:
trainer.evaluate()

{'eval_loss': 0.16001540422439575,
 'eval_f1': 81.2771,
 'eval_gen_len': 2.8311133879781423,
 'eval_runtime': 121.1962,
 'eval_samples_per_second': 48.318,
 'eval_steps_per_second': 6.04,
 'epoch': 2.0}

### Save the model on Huggingface Hub

In [74]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)

trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

('flan-t5-base-airline-sentiment-analysis/tokenizer_config.json',
 'flan-t5-base-airline-sentiment-analysis/special_tokens_map.json',
 'flan-t5-base-airline-sentiment-analysis/spiece.model',
 'flan-t5-base-airline-sentiment-analysis/added_tokens.json',
 'flan-t5-base-airline-sentiment-analysis/tokenizer.json')

events.out.tfevents.1713600095.0c854e8a1ac1.458.2:   0%|          | 0.00/456 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sudhanshusinghaiml/flan-t5-base-airline-sentiment-analysis/commit/f5cd9999501d684effbfb8d7e39306834ee99764', commit_message='End of training', commit_description='', oid='f5cd9999501d684effbfb8d7e39306834ee99764', pr_url=None, pr_revision=None, pr_num=None)

### Run Inference and Classification Report

In [71]:
from tqdm.auto import tqdm

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text = dataset['test']['text'][i]
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['label'][i])

  progress_bar.update(1)

  0%|          | 0/879 [00:00<?, ?it/s]

True

True

True

True

...

True

True

True

True

True

In [72]:
str_labels_list = []
for i in range(len(labels_list)): str_labels_list.append(str(labels_list[i]))

In [73]:
from sklearn.metrics import classification_report

report = classification_report(str_labels_list, predictions_list, zero_division=0)
print(report)

              precision    recall  f1-score   support

          -1       0.89      0.89      0.89       531
           0       0.69      0.64      0.67       191
           1       0.78      0.85      0.81       157

    accuracy                           0.83       879
   macro avg       0.79      0.80      0.79       879
weighted avg       0.83      0.83      0.83       879



### Load the fine tuned model from Hugging face hub

In [75]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("sudhanshusinghaiml/flan-t5-base-airline-sentiment-analysis")

model = AutoModelForSeq2SeqLM.from_pretrained("sudhanshusinghaiml/flan-t5-base-airline-sentiment-analysis")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [76]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

#### We can see that this model performs slightly better when compared to ML Algorithms such as Decision Tree, LDA and Random Forest